In [4]:
import sys
import random
import time

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
# import geopandas as gpd
import matplotlib.pyplot as plt

from datetime import datetime
# url = r'https://matokeo.necta.go.tz/results2021/csee/csee.htm'
url = r'https://matokeo.necta.go.tz/csee2022/index.htm'

In [5]:
pg = requests.get(url)
soup = bs(pg.content, 'html.parser')

In [6]:
table = soup.find_all('table')[2]
# table

In [7]:
links = table.find_all('a')

# links
s_links = []
for link in links:
    lintext = link.text.strip()
    if not lintext.startswith('P'):
#         print(lintext)
        s_links.append(link)
print(s_links[110])

<a href="results\s0259.htm">S0259 SAMARITAN GIRLS 
</a>


In [10]:

n_retry = 0
def get_region(lin):
    page = requests.get(lin)
    sp = bs(page.content, 'html.parser')
    reg = sp.find_all('table')[4].find_all('p')[1].text
    return reg
        
    

def fetch_results(lin, name):
    
    
    global n_retry
    try:
        #Bad HTML Tables
        #Will read 'CIVICS' info as headers
        ts = pd.read_html(lin, header = 0)
    #         print(len(ts))
        df = ts[1]
        school_gpa = float(ts[2].iloc[0,1].split(' ')[0])

        df['centre_ref'] = name.split(' ')[0].strip()
        df['centre_name'] = ' '.join(name.split(' ')[1:])
        df['region'] = get_region(lin)
        n_retry = 0
        school_sat = float(ts[-3].columns[2])
        school_regd = float(ts[-3].columns[0])
        subjs_df = ts[-1]
        math_df = subjs_df[subjs_df.CIVICS == 'BASIC MATHEMATICS']
        math_gpa = float(math_df.iloc[0,-2])
        math_sat = float(math_df.iloc[0,3])
        
        swahili_df = subjs_df[subjs_df.CIVICS == 'KISWAHILI']
        swahili_gpa = float(swahili_df.iloc[0,-2])
        swahili_sat = float(swahili_df.iloc[0,3])
        
        school_data = {
            'name': name,
            'school_gpa': school_gpa,
            'math_gpa': math_gpa,
            'school_regd': school_regd,
            'school_sat': school_sat,
            'math_sat': math_sat,
            'swahili_gpa': swahili_gpa,
            'swahili_sat': swahili_sat,
        }
#         print(school_data)
        return ts[1], school_data
    
    except:
        n_retry += 1
        time.sleep(3)
        if n_retry <=5:
            print(f'Retrying .... #{n_retry}')
            fetch_results(lin,name)
            
        return None, {}
    
    

###==================START HERE ===============================
res_tables = []
schools_data = []
milestone = 110
count = milestone
step = 10
n_links = len(links[milestone:1000])

for j in range(milestone, 1000 ,step):
    print(datetime.now())
    print('*---*   ' * 10 )
    print(f'Working from {j+1} to {j+step}')
    print('-'*50)
    for link in s_links[j:j+step]:
        t = random.randint(0,2)
#         time.sleep(t)
        lintext = link.text.strip()

        href = r'https://matokeo.necta.go.tz/csee2022/' + link.get('href').replace('\\','/')

    #     if lintext.startswith('S'):
        print('{0}. Fetching {1}'.format(count+1, lintext))
        print('\t' + href)
        results, school_data = fetch_results(href, lintext)
        if school_data == {}:
            count+=1
            continue
        res_tables.append(results)
        schools_data.append(school_data)

        count +=1
    my_df = pd.concat(res_tables)
    my_df.to_csv(r'G:\My Drive\consultancy\CSEE2022 Analysis\2022\csee{}_{}.csv'.format(j+1,j+step))
    res_tables = []
    
    
    schools_df = pd.DataFrame(schools_data)
    schools_df.to_csv(r'G:\My Drive\consultancy\CSEE2022 Analysis\2022\schools{}_{}.csv'.format(j+1,j+step))
    schools_data = []
    

2023-02-10 22:46:14.011159
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 111 to 120
--------------------------------------------------
111. Fetching S0259 SAMARITAN GIRLS
	https://matokeo.necta.go.tz/csee2022/results/s0259.htm
112. Fetching S0260 ST. ANN'S GIRLS SEMINARY
	https://matokeo.necta.go.tz/csee2022/results/s0260.htm
113. Fetching S0261 MLIMBA GIRLS HIGH
	https://matokeo.necta.go.tz/csee2022/results/s0261.htm
114. Fetching S0262 AMANI GIRLS
	https://matokeo.necta.go.tz/csee2022/results/s0262.htm
115. Fetching S0263 VISITATION GIRLS'
	https://matokeo.necta.go.tz/csee2022/results/s0263.htm
116. Fetching S0264 BARBRO-JOHANSSON
	https://matokeo.necta.go.tz/csee2022/results/s0264.htm
117. Fetching S0265 CORNELIUS
	https://matokeo.necta.go.tz/csee2022/results/s0265.htm
Retrying .... #1
Retrying .... #2
Retrying .... #3
Retrying .... #4
Retrying .... #5
118. Fetching S0266 REGINA MUNDI GIRLS
	https://matokeo.necta.go.tz/csee2022/results

188. Fetching S0367 KILOSA
	https://matokeo.necta.go.tz/csee2022/results/s0367.htm
189. Fetching S0368 IMBORU
	https://matokeo.necta.go.tz/csee2022/results/s0368.htm
190. Fetching S0371 LOMBETA
	https://matokeo.necta.go.tz/csee2022/results/s0371.htm
2023-02-10 22:49:07.050970
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 191 to 200
--------------------------------------------------
191. Fetching S0372 KIRUA
	https://matokeo.necta.go.tz/csee2022/results/s0372.htm
192. Fetching S0373 ITOPE
	https://matokeo.necta.go.tz/csee2022/results/s0373.htm
193. Fetching S0374 LUPATA
	https://matokeo.necta.go.tz/csee2022/results/s0374.htm
194. Fetching S0375 JUMUIYA
	https://matokeo.necta.go.tz/csee2022/results/s0375.htm
195. Fetching S0376 SONI SEMINARY
	https://matokeo.necta.go.tz/csee2022/results/s0376.htm
196. Fetching S0378 HEGONGO HOLY CROSS
	https://matokeo.necta.go.tz/csee2022/results/s0378.htm
197. Fetching S0380 UCHAMA
	https://matokeo.necta.g

266. Fetching S0474 KISOMACHI
	https://matokeo.necta.go.tz/csee2022/results/s0474.htm
267. Fetching S0475 KIDIA
	https://matokeo.necta.go.tz/csee2022/results/s0475.htm
268. Fetching S0476 MWANGAZA
	https://matokeo.necta.go.tz/csee2022/results/s0476.htm
269. Fetching S0477 CHIMALA
	https://matokeo.necta.go.tz/csee2022/results/s0477.htm
270. Fetching S0478 KIPOKE
	https://matokeo.necta.go.tz/csee2022/results/s0478.htm
2023-02-10 22:51:58.129487
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 271 to 280
--------------------------------------------------
271. Fetching S0479 NRONGA
	https://matokeo.necta.go.tz/csee2022/results/s0479.htm
272. Fetching S0480 NSOO
	https://matokeo.necta.go.tz/csee2022/results/s0480.htm
273. Fetching S0482 ILUHYA
	https://matokeo.necta.go.tz/csee2022/results/s0482.htm
274. Fetching S0484 MKINGA
	https://matokeo.necta.go.tz/csee2022/results/s0484.htm
275. Fetching S0485 MAJENGO
	https://matokeo.necta.go.tz/csee2022/r

346. Fetching S0577 AL-HIKMA GIRLS
	https://matokeo.necta.go.tz/csee2022/results/s0577.htm
347. Fetching S0578 TAQWA
	https://matokeo.necta.go.tz/csee2022/results/s0578.htm
348. Fetching S0579 KATUNGURU
	https://matokeo.necta.go.tz/csee2022/results/s0579.htm
349. Fetching S0580 IGOWOLE
	https://matokeo.necta.go.tz/csee2022/results/s0580.htm
350. Fetching S0581 ILEJE
	https://matokeo.necta.go.tz/csee2022/results/s0581.htm
2023-02-10 22:54:33.246369
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 351 to 360
--------------------------------------------------
351. Fetching S0582 UNUNIO ISLAMIC HIGH SCHOOL
	https://matokeo.necta.go.tz/csee2022/results/s0582.htm
352. Fetching S0583 MAZINYUNGU
	https://matokeo.necta.go.tz/csee2022/results/s0583.htm
353. Fetching S0584 LWANDAI
	https://matokeo.necta.go.tz/csee2022/results/s0584.htm
354. Fetching S0585 MUNANILA
	https://matokeo.necta.go.tz/csee2022/results/s0585.htm
355. Fetching S0586 KAISHO
	https

425. Fetching S0669 UTETE
	https://matokeo.necta.go.tz/csee2022/results/s0669.htm
426. Fetching S0670 NYANG'HWALE
	https://matokeo.necta.go.tz/csee2022/results/s0670.htm
427. Fetching S0671 UBUNGO ISLAMIC HIGH SCHOOL
	https://matokeo.necta.go.tz/csee2022/results/s0671.htm
428. Fetching S0672 MKWAKWANI
	https://matokeo.necta.go.tz/csee2022/results/s0672.htm
429. Fetching S0675 MBUGWE
	https://matokeo.necta.go.tz/csee2022/results/s0675.htm
430. Fetching S0676 MABIRA
	https://matokeo.necta.go.tz/csee2022/results/s0676.htm
2023-02-10 22:57:08.211289
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 431 to 440
--------------------------------------------------
431. Fetching S0677 TANDAHIMBA
	https://matokeo.necta.go.tz/csee2022/results/s0677.htm
432. Fetching S0678 MUGANGO
	https://matokeo.necta.go.tz/csee2022/results/s0678.htm
433. Fetching S0679 KIMANDOLU
	https://matokeo.necta.go.tz/csee2022/results/s0679.htm
434. Fetching S0680 MWAZYE
	https:/

...

664. Fetching S0935 MWANDOYA
	https://matokeo.necta.go.tz/csee2022/results/s0935.htm
665. Fetching S0936 KALIUA
	https://matokeo.necta.go.tz/csee2022/results/s0936.htm
666. Fetching S0938 MBEZI BEACH
	https://matokeo.necta.go.tz/csee2022/results/s0938.htm
667. Fetching S0939 NKOKASHU
	https://matokeo.necta.go.tz/csee2022/results/s0939.htm
668. Fetching S0940 NDAGO
	https://matokeo.necta.go.tz/csee2022/results/s0940.htm
669. Fetching S0941 MADIVENI
	https://matokeo.necta.go.tz/csee2022/results/s0941.htm
670. Fetching S0943 IFATI
	https://matokeo.necta.go.tz/csee2022/results/s0943.htm
2023-02-10 23:04:42.532176
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 671 to 680
--------------------------------------------------
671. Fetching S0945 KWIZU
	https://matokeo.necta.go.tz/csee2022/results/s0945.htm
672. Fetching S0947 DR. OLSEN
	https://matokeo.necta.go.tz/csee2022/results/s0947.htm
673. Fetching S0948 BUMANGI
	https://matokeo.necta.go.tz/cs

743. Fetching S1032 ITIGI
	https://matokeo.necta.go.tz/csee2022/results/s1032.htm
744. Fetching S1033 AMANI ABEID KARUME
	https://matokeo.necta.go.tz/csee2022/results/s1033.htm
745. Fetching S1034 ITILIMA
	https://matokeo.necta.go.tz/csee2022/results/s1034.htm
746. Fetching S1035 KWEDIBOMA
	https://matokeo.necta.go.tz/csee2022/results/s1035.htm
747. Fetching S1036 LUBANDA
	https://matokeo.necta.go.tz/csee2022/results/s1036.htm
748. Fetching S1037 AYA
	https://matokeo.necta.go.tz/csee2022/results/s1037.htm
749. Fetching S1038 J.K. NYERERE
	https://matokeo.necta.go.tz/csee2022/results/s1038.htm
750. Fetching S1039 GITTING
	https://matokeo.necta.go.tz/csee2022/results/s1039.htm
2023-02-10 23:07:00.232449
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 751 to 760
--------------------------------------------------
751. Fetching S1041 LONGOI
	https://matokeo.necta.go.tz/csee2022/results/s1041.htm
752. Fetching S1042 MUNGAA
	https://matokeo.necta.

822. Fetching S1120 MOITA
	https://matokeo.necta.go.tz/csee2022/results/s1120.htm
823. Fetching S1121 MAWELLA
	https://matokeo.necta.go.tz/csee2022/results/s1121.htm
824. Fetching S1122 CHIKANAMLILO
	https://matokeo.necta.go.tz/csee2022/results/s1122.htm
825. Fetching S1123 MZENGA
	https://matokeo.necta.go.tz/csee2022/results/s1123.htm
826. Fetching S1124 KIGARAMA
	https://matokeo.necta.go.tz/csee2022/results/s1124.htm
827. Fetching S1125 ENYOITO
	https://matokeo.necta.go.tz/csee2022/results/s1125.htm
828. Fetching S1126 MINZIRO
	https://matokeo.necta.go.tz/csee2022/results/s1126.htm
829. Fetching S1127 BUGANDO
	https://matokeo.necta.go.tz/csee2022/results/s1127.htm
830. Fetching S1128 IMALILO
	https://matokeo.necta.go.tz/csee2022/results/s1128.htm
2023-02-10 23:09:18.395665
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 831 to 840
--------------------------------------------------
831. Fetching S1129 IGUGUNO
	https://matokeo.necta.go.tz/c

902. Fetching S1208 KISHOGO
	https://matokeo.necta.go.tz/csee2022/results/s1208.htm
903. Fetching S1209 SANZA
	https://matokeo.necta.go.tz/csee2022/results/s1209.htm
904. Fetching S1211 KATESH
	https://matokeo.necta.go.tz/csee2022/results/s1211.htm
905. Fetching S1212 IKOMA
	https://matokeo.necta.go.tz/csee2022/results/s1212.htm
906. Fetching S1215 ENGUSERO
	https://matokeo.necta.go.tz/csee2022/results/s1215.htm
907. Fetching S1216 MIPA
	https://matokeo.necta.go.tz/csee2022/results/s1216.htm
908. Fetching S1217 MWASAYI
	https://matokeo.necta.go.tz/csee2022/results/s1217.htm
909. Fetching S1219 FUKUCHANI
	https://matokeo.necta.go.tz/csee2022/results/s1219.htm
910. Fetching S1220 MSAMALA
	https://matokeo.necta.go.tz/csee2022/results/s1220.htm
2023-02-10 23:11:36.118303
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 911 to 920
--------------------------------------------------
911. Fetching S1221 MUHANGE
	https://matokeo.necta.go.tz/csee2022/

982. Fetching S1299 MTULE
	https://matokeo.necta.go.tz/csee2022/results/s1299.htm
983. Fetching S1300 KWAMTIPURA
	https://matokeo.necta.go.tz/csee2022/results/s1300.htm
984. Fetching S1301 MAHONDA
	https://matokeo.necta.go.tz/csee2022/results/s1301.htm
985. Fetching S1303 UROA
	https://matokeo.necta.go.tz/csee2022/results/s1303.htm
986. Fetching S1304 CHANJAMJAWIRI
	https://matokeo.necta.go.tz/csee2022/results/s1304.htm
987. Fetching S1305 MINUNGWINI
	https://matokeo.necta.go.tz/csee2022/results/s1305.htm
988. Fetching S1306 MKANYAGENI
	https://matokeo.necta.go.tz/csee2022/results/s1306.htm
989. Fetching S1307 MAENDELEO NGWACHANI
	https://matokeo.necta.go.tz/csee2022/results/s1307.htm
990. Fetching S1308 TUMBE
	https://matokeo.necta.go.tz/csee2022/results/s1308.htm
2023-02-10 23:13:43.283349
*---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   *---*   
Working from 991 to 1000
--------------------------------------------------
991. Fetching S1309 VITONGOJI
	https://m

In [ ]:
pd.read_html?

In [ ]:
ts[-1]

In [ ]:
len(res_tables)

In [ ]:
sys.getsizeof(res_tables)

In [ ]:
my_df = pd.concat(res_tables)

In [ ]:
sys.getsizeof(my_df)

In [ ]:
ts = pd.read_html(href, header = 0)
df = ts[-2]
# df['centre_ref'] = name.split(' ')[0].strip()
# df['centre_name'] = ' '.join(name.split(' ')[1:])
print(ts[3])

In [ ]:
soup

In [ ]:
ts[2]

In [ ]:
lin = r'https://necta.go.tz/results/2020/csee/results/s0194.htm'
page = requests.get(lin)
sp = bs(page.content, 'html.parser')
reg = sp.find_all('table')[4].find_all('p')[1].text


In [ ]:

tz = pd.read_html(lin)

In [ ]:
tz[4]

In [ ]:
count = 0
for t in tss:
    print(t)
    print('*'*40)
    print(count)
    count+=1

In [ ]:
for i in range(0,10,  2):
    print(i)

In [ ]:
links

In [ ]:
for link in links:
    lintext = link.text.strip()
    if not lintext.startswith('P'):
        print(lintext)

In [ ]:
import random
[random.randint(0,2) for i in range(10)]